Подготовка базы данных

In [1]:
import pandas as pd
import threading
import sys

In [2]:
data = pd.read_csv('Raw_data.csv')

In [3]:
data.shape

(371583, 21)

In [4]:
data = data.query("businessman_flag == 0")
data = data.drop(['term','businessman_flag'], axis=1)

In [5]:
data = data.fillna({'start_date': -1, 'fact_close_date': -1, 'card_type_name':-1})

In [6]:
data = data.astype({'start_date': int, 'fact_close_date': int , 'purchase_sum':int ,'purchase_sum':int ,'contract_sum':int , 'current_balance_avg_sum':int , 'current_balance_sum':int, 'current_debit_turn_sum':int, 'current_credit_turn_sum':int})

In [7]:
data.shape

(371583, 19)

In [8]:
data = data.drop_duplicates() # Избавление от дубликатов записей

Поиск валидных записей

In [9]:
listUniqueCardID = data.card_id.unique()

In [10]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [11]:
def th_get_ActualUserData(gl_DataFrame, listUniqueCardID, intIndex):
    local_UsersData = pd.DataFrame()
    for i in range(len(listUniqueCardID)):
        if float(i%100) == 0:
            print(i)
            
        strPDCall = f"card_id == '{listUniqueCardID[i]}'" # Запрос

        #f_Data = data.query(strPDCall)
        #print( listUniqueCardID[i] , '-', f_Data.shape)

        local_Data = data.query(strPDCall).drop_duplicates()
        #print( listUniqueCardID[i] , '---', local_Data.shape,'\n')

        if (local_Data.shape[0] != 1):
              
            maxDebet =  local_Data['current_debit_turn_sum'].max()
            maxCredet = local_Data['current_credit_turn_sum'].max()
            maxPurchaseCount = local_Data['purchase_count'].max()
            
            thisDB = local_Data.query(f"current_debit_turn_sum == {maxDebet} & current_credit_turn_sum == {maxCredet} & purchase_count == {maxPurchaseCount}")
            
        else:
            thisDB = local_Data
        
        #print(i, "-", thisDB.shape, "-", listUniqueCardID[i],'\n')

        local_UsersData = pd.concat([local_UsersData, thisDB]) 
           
        #print(local_UsersData.shape)

    local_UsersData.to_csv(f'JSON/New_CSV_{intIndex}.csv')

    gl_DataFrame = pd.concat([gl_DataFrame, local_UsersData]) 
    print(gl_DataFrame.shape)

    print(f'Thread {intIndex} is done !')
    sys.exit

In [12]:
listParts = getlistEqualPartsOfNumberParts(len(listUniqueCardID),int(len(listUniqueCardID)/800))
listThreads = []
intCount = 0
gl_ActualUserData = pd.DataFrame()

intOldTreadsCount = len(threading.enumerate())

for i in range(len(listParts)):

    if listParts[i] == 0 :
        break
    print(f'Thread {i} from {intCount} to {intCount+listParts[i]} is start - {listParts[i]}')
    listThreads.append( threading.Thread( name=f'Thread_{i}', target=th_get_ActualUserData, args=(gl_ActualUserData, listUniqueCardID[ intCount: intCount+listParts[i] ],i , ) ))
    intCount += listParts[i]+1
    listThreads[i].start()

Thread 0 from 0 to 502 is start - 502
0
Thread 1 from 503 to 1005 is start - 502
0Thread 2 from 1006 to 1508 is start - 502

0
Thread 3 from 1509 to 2011 is start - 502
0
Thread 4 from 2012 to 2514 is start - 502
0Thread 5 from 2515 to 3017 is start - 502
0
Thread 6 from 3018 to 3520 is start - 502

0
Thread 7 from 3521 to 4023 is start - 502
0
Thread 8 from 4024 to 4526 is start - 502
0
Thread 9 from 4527 to 5029 is start - 502
0
Thread 10 from 5030 to 5532 is start - 502
0Thread 11 from 5533 to 6035 is start - 502

0
Thread 12 from 6036 to 6538 is start - 502
0
Thread 13 from 6539 to 7041 is start - 502
0
Thread 14 from 7042 to 7544 is start - 502
0
Thread 15 from 7545 to 8047 is start - 502
0
Thread 16 from 8048 to 8550 is start - 502
0
Thread 17 from 8551 to 9053 is start - 502
0
Thread 18 from 9054 to 9556 is start - 502
0
Thread 19 from 9557 to 10059 is start - 502
0
Thread 20 from 10060 to 10562 is start - 502
0Thread 21 from 10563 to 11065 is start - 502

0
Thread 22 from 11066

In [13]:
while(True):
    if intOldTreadsCount == len(threading.enumerate()) : break

gl_ActualUserData = pd.DataFrame()
count = 0

for i in range(len(listParts)):
    gl_ActualUserData = pd.concat([gl_ActualUserData, pd.read_csv(f'JSON/New_CSV_{i}.csv').drop_duplicates()]) 
    count += pd.read_csv(f'JSON/New_CSV_{i}.csv').shape[0]
gl_ActualUserData.shape

FileNotFoundError: [Errno 2] No such file or directory: 'JSON/New_CSV_0.csv'

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100100

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100100

100
100
100
100
200
200
200
200
200
200
200
200
200
200
200
200
200
200200

200
200
200
200
200
200
200200

200
200
200
200200
200
200

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
300
300
300
300
300300

300
300
300
300
300
300300

300
300
300300
300

300
300300

300
300
300300
300

300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300300

300


In [ ]:
gl_ActualUserData.drop(['Unnamed: 0'], axis=1)

In [ ]:
gl_ActualUserData.to_csv('RTG_CSV.csv')